# 17.2 Zadanie Random Forest

#### Import niezbednych bibliotek i paczek

In [1]:
import numpy as np
import pandas as pd
import string
import nltk
import itertools
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\piotr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\piotr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\piotr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Definicje wszystkich funkcji

In [2]:
def remove_puncation(text):
    cleaned = ''.join([word for word in text if word not in string.punctuation])
    return cleaned
def tokenize(text):
    # Usunięcie wielkich liter
    clean_text = text.lower()
    # Tokenizacja
    tokenized_text = nltk.word_tokenize(clean_text)
    return tokenized_text
def remove_stopwords(text):
    without_stopwords = [word for word in text if word not in stopwords]
    return without_stopwords
def stemming(text):
    stemmed_words = [stemmer.stem(word) for word in text]
    return stemmed_words
def lemmatizing(text):
    lemmatized_words = [lemmater.lemmatize(word) for word in text]
    return lemmatized_words

#### Import pliku z danymi i jego formatowanie

In [3]:
spam_dataset = pd.read_csv('spam.csv', encoding = "ISO-8859-1", usecols=[0, 1], names=['Spam', 'Text'],
                           skiprows=1)
spam_dataset['Spam'] = spam_dataset['Spam'].replace(['ham', 'spam'], [0, 1])
spam_dataset

C:\Users\piotr\AppData\Local\Temp\ipykernel_31360\1996902043.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  spam_dataset['Spam'] = spam_dataset['Spam'].replace(['ham', 'spam'], [0, 1])


,Spam,Text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


#### Obróbka danych - czyszczenie, tokenizacja, usuwanie stop words, Stemmed, Lametyzcja

In [4]:
spam_dataset['Cleaned_Text'] = spam_dataset['Text'].apply(lambda x: remove_puncation(x))
spam_dataset['Tokenized_Text'] = spam_dataset['Cleaned_Text'].apply(lambda x: tokenize(x))
stopwords = nltk.corpus.stopwords.words("english")
spam_dataset['WithoutStop_Text'] = spam_dataset['Tokenized_Text'].apply(lambda x: remove_stopwords(x))
stemmer = nltk.PorterStemmer()
spam_dataset['Stemmed_Text'] = spam_dataset['WithoutStop_Text'].apply(lambda x: stemming(x))
lemmater = nltk.WordNetLemmatizer()
spam_dataset['Lemmatized_Text'] = spam_dataset['WithoutStop_Text'].apply(lambda x: lemmatizing(x))
spam_dataset

,Spam,Text,Cleaned_Text,Tokenized_Text,WithoutStop_Text,Stemmed_Text,Lemmatized_Text
0,0,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n...","[go, jurong, point, crazi, avail, bugi, n, gre...","[go, jurong, point, crazy, available, bugis, n..."
1,0,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]","[ok, lar, joke, wif, u, oni]","[ok, lar, joking, wif, u, oni]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entri, 2, wkli, comp, win, fa, cup, fin...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,0,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]","[u, dun, say, earli, hor, u, c, alreadi, say]","[u, dun, say, early, hor, u, c, already, say]"
4,0,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t...","[nah, dont, think, goe, usf, live, around, tho...","[nah, dont, think, go, usf, life, around, though]"
...,...,...,...,...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,This is the 2nd time we have tried 2 contact u...,"[this, is, the, 2nd, time, we, have, tried, 2,...","[2nd, time, tried, 2, contact, u, u, å£750, po...","[2nd, time, tri, 2, contact, u, u, å£750, poun...","[2nd, time, tried, 2, contact, u, u, å£750, po..."
5568,0,Will Ì_ b going to esplanade fr home?,Will Ì b going to esplanade fr home,"[will, ì, b, going, to, esplanade, fr, home]","[ì, b, going, esplanade, fr, home]","[ì, b, go, esplanad, fr, home]","[ì, b, going, esplanade, fr, home]"
5569,0,"Pity, * was in mood for that. So...any other s...",Pity was in mood for that Soany other suggest...,"[pity, was, in, mood, for, that, soany, other,...","[pity, mood, soany, suggestions]","[piti, mood, soani, suggest]","[pity, mood, soany, suggestion]"
5570,0,The guy did some bitching but I acted like i'd...,The guy did some bitching but I acted like id ...,"[the, guy, did, some, bitching, but, i, acted,...","[guy, bitching, acted, like, id, interested, b...","[guy, bitch, act, like, id, interest, buy, som...","[guy, bitching, acted, like, id, interested, b..."


#### Wektoryzacja tekstu i rozdzielenie na X, y

In [5]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(spam_dataset['Lemmatized_Text'].apply(lambda x: ' '.join(x)))
y = spam_dataset['Spam']
feature_names = tfidf.vocabulary_.keys()

#### Rozdzielenie danych na zbiory treningowe oraz testowe

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [7]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8663677130044843

#### Określenie które słowa są najwazniejsze oraz określenie ich wazności

In [8]:
importances = clf.feature_importances_
feature_imp_df = pd.DataFrame({'Word': feature_names, 'Word Importance': importances}).sort_values('Word Importance', ascending=False) 
print(feature_imp_df)

               Word  Word Importance
7618         pimple         0.037414
5193         mirror         0.034702
2453          weird         0.033022
7368        trishul         0.029568
7679        hostile         0.026637
...             ...              ...
2980         marley         0.000000
2979  stereophonics         0.000000
2978        lastest         0.000000
2977        charles         0.000000
8842       bitching         0.000000

[8843 rows x 2 columns]


#### Stworzenie listy z złów których wazność jest większa niż 0.001

In [9]:
imp_word_lis = feature_imp_df[feature_imp_df['Word Importance']>0.001].iloc[:,0].to_list()
imp_word_lis

['pimple',
 'mirror',
 'weird',
 'trishul',
 'hostile',
 'mah',
 'wood',
 'canal',
 'belovd',
 'nvq',
 'incorrect',
 'videopic',
 'ringtone',
 'shopping',
 '2814032',
 'jstfrnd',
 'keyword',
 'versus',
 '08718738002',
 'sp',
 'meive',
 '25p',
 'frnt',
 '5p',
 'basket',
 'twat',
 'prescription',
 'bt',
 'compensation',
 'bbdpooja',
 '600',
 '09050280520',
 'cnn',
 'starwars3',
 '9yt',
 'enjoy',
 'loud',
 'check',
 'adult',
 'hard',
 '830',
 'srsly',
 'texas',
 'direct',
 'ringtonefrom',
 'cutefrnd',
 'blanket',
 'matra',
 'startedindia',
 'fair',
 'knackered',
 'dent',
 'warned',
 'b4190604',
 'real',
 'horse',
 '1yf',
 'oredi',
 'tension',
 'jess',
 'arsenal',
 'owed',
 'manual',
 'bedroom',
 'october',
 'daddy',
 'possiblehope',
 '2untamed',
 '3wife',
 '8am',
 'jesus',
 'tomo',
 'worth',
 'skirt',
 'ups',
 'alexs',
 'local',
 'learn',
 'girld',
 'lip',
 'studying',
 'wwwsantacallingcom',
 'ceiling',
 'morning',
 'hasbroin',
 'hi',
 'oooooh',
 'wendy',
 'franxx',
 'copy',
 'machiany',


#### Tworzymu nową kolumnę w spam_dataset gdzie będziemy mieć wyłącznie wazne słowa po lametyzacji

In [10]:
def important_only(text):
    important_only = [word for word in text if word in imp_word_lis]
    return important_only

In [11]:
spam_dataset['Important_Text'] = spam_dataset['Lemmatized_Text'].apply(lambda x: important_only(x))
spam_dataset['Important_Text']

0           []
1           []
2           []
3           []
4       [life]
         ...  
5567        []
5568        []
5569        []
5570        []
5571        []
Name: Important_Text, Length: 5572, dtype: object

#### Przeprowadzamy wektoryzacje teksty którego ważnść jest większa niż 0.001, rozdzielamy dane 

In [12]:
tfidf_imp =  TfidfVectorizer()
X = tfidf_imp.fit_transform(spam_dataset['Important_Text'].apply(lambda x: ' '.join(x)))
y = spam_dataset['Spam']
X_train_imp, X_test_imp, y_train_imp, y_test_imp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#### Tworzymy GridSearch dla najwazniejszych parametrów klasyfikatora, szkolimy go z najlepszymi parametrami i odczytujemy wynik. 

In [13]:
random_forest = RandomForestClassifier( n_jobs=-1)
params_rf = {'max_depth': [ 15, 20, 25],
             'min_samples_leaf': [2, 3, 5],
             'n_estimators': [  25, 30, 40 ],
             'min_samples_split': [2, 3, 5]}

rf_gridsearch = GridSearchCV(random_forest,
                             params_rf,
                             scoring='f1_micro',
                             cv=5,
                             verbose=10, n_jobs=-1)
rf_gridsearch.fit(X_train_imp, y_train_imp)
print('\nBest hyperparameter:', rf_gridsearch.best_params_)
model = rf_gridsearch.best_estimator_
model.fit(X_train_imp, y_train_imp)
predictions_test = model.predict(X_test_imp)
predictions_train = model.predict(X_train_imp)
f1_score_train = f1_score(y_train_imp, predictions_train, average='micro').round(4)
f1_score_test = f1_score(y_test_imp, predictions_test, average='micro').round(4)
print("LogisticRegression F1_score Train:", f1_score_train)
print("LogisticRegression F1_score Test:", f1_score_test)

Fitting 5 folds for each of 81 candidates, totalling 405 fits

Best hyperparameter: {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 40}
LogisticRegression F1_score Train: 0.8759
LogisticRegression F1_score Test: 0.8762


#### Udało się poprwić wynik F1 Score